In [1]:
import pandas
import seaborn
import numpy

import sklearn.impute
import sklearn.preprocessing

%matplotlib inline

# Instructions

- Read training, test, and live data from CSV files
- Then, identify which columns are numerical and which columns are categorical
    - Note that some features that appear numeric might actually be categorical
- Assess whether the data is balance or imbalance

In [2]:
data_train = pandas.read_csv('../data/credit.train.csv').set_index('id')
data_train

checking_status  duration credit_history              purpose  \
id                                                                  
495              <0      24.0  existing paid  furniture/equipment   
296     no checking      12.0  existing paid             used car   
1          0<=X<200      48.0  existing paid             radio/tv   
38            >=200      10.0  existing paid   domestic appliance   
568        0<=X<200      48.0  existing paid             radio/tv   
..              ...       ...            ...                  ...   
836     no checking      12.0  existing paid             radio/tv   
192        0<=X<200      27.0  existing paid             business   
630              <0      24.0  existing paid             radio/tv   
560              <0      24.0  existing paid             used car   
685     no checking      60.0  existing paid              new car   

     credit_amount    savings_status employment  installment_commitment  \
id                                                                        
495         2996.0  no known savings     1<=X<4                       2   
296         4675.0  no known savings         <1                       1   
1           5951.0              <100     1<=X<4                       2   
38          1225.0              <100     1<=X<4                       2   
568         3979.0  no known savings     4<=X<7                       4   
..             ...               ...        ...                     ...   
836          886.0  no known savings     1<=X<4                       4   
192         3915.0              <100     1<=X<4                       4   
630         3660.0              <100     1<=X<4                       2   
560         2964.0  no known savings        >=7                       4   
685         6527.0  no known savings     1<=X<4                       4   

        personal_status other_parties  ...  property_magnitude age  \
id                                     ...                           
495        male mar/wid          none  ...                 car  20   
296  female div/dep/mar          none  ...                 car  20   
1    female div/dep/mar          none  ...         real estate  22   
38          male single          none  ...                 car  37   
568         male single          none  ...                 car  41   
..                  ...           ...  ...                 ...  ..   
836  female div/dep/mar          none  ...                 car  21   
192         male single          none  ...                 car  36   
630  female div/dep/mar          none  ...                 car  28   
560         male single          none  ...   no known property  49   
685         male single          none  ...   no known property  34   

     other_payment_plans   housing existing_credits      job num_dependents  \
id                                                                            
495                 none       own              1.0  skilled              1   
296                 none      rent              1.0  skilled              1   
1                   none       own              1.0  skilled              1   
38                  none       own              1.0  skilled              1   
568                 none       own              2.0  skilled              2   
..                   ...       ...              ...      ...            ...   
836                 none       own              1.0  skilled              1   
192                 none       own              1.0  skilled              2   
630                 none       own              1.0  skilled              1   
560                 bank  for free              1.0  skilled              2   
685                 none  for free              1.0  skilled              2   

     own_telephone foreign_worker label  
id                                       
495           none            yes   bad  
296           none            yes  good  
1             none     

In [3]:
data_test = pandas.read_csv('../data/credit.test.csv').set_index('id')
data_test

checking_status  duration                  credit_history  \
id                                                              
271     no checking      12.0  critical/other existing credit   
726     no checking      15.0  critical/other existing credit   
31               <0      24.0                   existing paid   
343        0<=X<200      18.0                   existing paid   
605              <0      24.0                        all paid   
..              ...       ...                             ...   
75               <0      12.0  critical/other existing credit   
454              <0      24.0                   existing paid   
468     no checking      33.0              delayed previously   
386     no checking      18.0                   existing paid   
492     no checking       6.0  critical/other existing credit   

                 purpose  credit_amount    savings_status employment  \
id                                                                     
271  furniture/equipment         1402.0       500<=X<1000     4<=X<7   
726             radio/tv         1316.0       500<=X<1000     1<=X<4   
31   furniture/equipment         4020.0              <100     1<=X<4   
343             business         4439.0              <100        >=7   
605  furniture/equipment         2828.0       500<=X<1000     1<=X<4   
..                   ...            ...               ...        ...   
75              used car         1526.0              <100        >=7   
454              new car         4817.0              <100     4<=X<7   
468             business         2764.0              <100     1<=X<4   
386             radio/tv         1126.0  no known savings         <1   
492             radio/tv         1237.0        100<=X<500     1<=X<4   

     installment_commitment     personal_status other_parties  ...  \
id                                                             ...   
271                       3  female div/dep/mar          none  ...   
726                       2        male mar/wid          none  ...   
31                        2         male single          none  ...   
343                       1         male single  co applicant  ...   
605                       4         male single          none  ...   
..                      ...                 ...           ...  ...   
75                        4         male single          none  ...   
454                       2         male single  co applicant  ...   
468                       2  female div/dep/mar          none  ...   
386                       4  female div/dep/mar          none  ...   
492                       1  female div/dep/mar          none  ...   

     property_magnitude age  other_payment_plans   housing existing_credits  \
id                                                                            
271                 car  37                 none      rent              1.0   
726      life insurance  47                 none       own              2.0   
31                  car  27               stores       own              1.0   
343         real estate  33                 bank       own              1.0   
605         real estate  22               stores       own              1.0   
..                  ...  ..                  ...       ...              ...   
75    no known property  66                 none  for free              2.0   
454      life insurance  31                 none       own              1.0   
468                 car  26                 none       own              2.0   
386         real estate  21                 none      rent              1.0   
492      life insurance  27                 none       own              2.0   

                           job num_dependents  own_telephone foreign_worker  \
id                                                                            
271                    skilled              1            yes            yes   
726         unskilled resident              1

In [4]:
data_live = pandas.read_csv('../data/credit.live.csv').set_index('id')
data_live

checking_status  duration                  credit_history  \
id                                                              
271     no checking      12.0  critical/other existing credit   
726     no checking      15.0  critical/other existing credit   
31               <0      24.0                   existing paid   
343        0<=X<200      18.0                   existing paid   
605              <0      24.0                        all paid   
..              ...       ...                             ...   
725     no checking       6.0  critical/other existing credit   
747              <0      12.0                   existing paid   
534     no checking      24.0                   existing paid   
97         0<=X<200      18.0                   existing paid   
268              <0      14.0                   existing paid   

                 purpose  credit_amount    savings_status employment  \
id                                                                     
271  furniture/equipment         1402.0       500<=X<1000     4<=X<7   
726             radio/tv         1316.0       500<=X<1000     1<=X<4   
31   furniture/equipment         4020.0              <100     1<=X<4   
343             business         4439.0              <100        >=7   
605  furniture/equipment         2828.0       500<=X<1000     1<=X<4   
..                   ...            ...               ...        ...   
725              new car          250.0            >=1000     1<=X<4   
747              new car         1274.0              <100         <1   
534             radio/tv         3105.0  no known savings         <1   
97              business         2622.0        100<=X<500     1<=X<4   
268              new car         8978.0              <100        >=7   

     installment_commitment     personal_status other_parties  \
id                                                              
271                       3  female div/dep/mar          none   
726                       2        male mar/wid          none   
31                        2         male single          none   
343                       1         male single  co applicant   
605                       4         male single          none   
..                      ...                 ...           ...   
725                       2  female div/dep/mar          none   
747                       3  female div/dep/mar          none   
534                       4         male single          none   
97                        4         male single          none   
268                       1        male div/sep          none   

     residence_since property_magnitude  age other_payment_plans housing  \
id                                                                         
271              4.0                car   37                none    rent   
726              2.0     life insurance   47                none     own   
31               2.0                car   27              stores     own   
343              1.0        real estate   33                bank     own   
605              4.0        real estate   22              stores     own   
..               ...                ...  ...                 ...     ...   
725              2.0        real estate   41                bank     own   
747              1.0        real estate   37                none     own   
534              2.0                car   25                none     own   
97               4.0                car   34                none     own   
268              4.0     life insurance   45                none     own   

     existing_credits                        job  num_dependents  \
id                                                                 
271               1.0                    skilled               1   
726               2.0         unskilled resident               1   
31                1.0                    skilled               1   
343               1.0  high qualif/self emp/mg

In [5]:
data_train.dtypes

checking_status            object
duration                  float64
credit_history             object
purpose                    object
credit_amount             float64
savings_status             object
employment                 object
installment_commitment      int64
personal_status            object
other_parties              object
residence_since           float64
property_magnitude         object
age                         int64
other_payment_plans        object
housing                    object
existing_credits          float64
job                        object
num_dependents              int64
own_telephone              object
foreign_worker             object
label                      object
dtype: object

In [6]:
categorical_features = [
    'checking_status', 
    'credit_history', 
    'purpose', 
    'savings_status',
    'employment', 
    'personal_status', 
    'other_parties', 
    'property_magnitude',
    'other_payment_plans', 
    'housing', 
    'job', 
    'own_telephone',
    'foreign_worker',
    'installment_commitment', 
    'residence_since', 
    'existing_credits', 
    'num_dependents'
]

for feature in categorical_features:
    possible_values = data_train[feature].unique()
    print(f"{feature:20s} {possible_values.size:2d} {possible_values}")

checking_status       5 ['<0' 'no checking' '0<=X<200' '>=200' nan]
credit_history        5 ['existing paid' 'critical/other existing credit' 'delayed previously'
 'all paid' 'no credits/all paid']
purpose              11 ['furniture/equipment' 'used car' 'radio/tv' 'domestic appliance'
 'new car' 'education' 'business' 'retraining' 'repairs' 'other' nan]
savings_status        5 ['no known savings' '<100' '>=1000' '100<=X<500' '500<=X<1000']
employment            5 ['1<=X<4' '<1' '4<=X<7' '>=7' 'unemployed']
personal_status       5 ['male mar/wid' 'female div/dep/mar' 'male single' 'male div/sep' nan]
other_parties         3 ['none' 'guarantor' 'co applicant']
property_magnitude    5 ['car' 'real estate' 'life insurance' 'no known property' nan]
other_payment_plans   3 ['none' 'bank' 'stores']
housing               3 ['own' 'rent' 'for free']
job                   4 ['skilled' 'unskilled resident' 'unemp/unskilled non res'
 'high qualif/self emp/mgmt']
own_telephone         3 ['none' '

In [7]:
data_train['label'].value_counts()

label
good    513
bad     235
Name: count, dtype: int64

In [8]:
data_test['label'].value_counts()

label
good    185
bad      65
Name: count, dtype: int64

# Instructions
- Investigate whether the dataset contains any missing data

In [9]:
data_train.isna().sum(axis='index')

checking_status           2
duration                  1
credit_history            0
purpose                   2
credit_amount             1
savings_status            0
employment                0
installment_commitment    0
personal_status           2
other_parties             0
residence_since           1
property_magnitude        1
age                       0
other_payment_plans       0
housing                   0
existing_credits          1
job                       0
num_dependents            0
own_telephone             1
foreign_worker            1
label                     0
dtype: int64

In [10]:
data_test.isna().sum(axis='index')

checking_status           1
duration                  0
credit_history            0
purpose                   0
credit_amount             0
savings_status            0
employment                0
installment_commitment    0
personal_status           0
other_parties             0
residence_since           0
property_magnitude        0
age                       0
other_payment_plans       0
housing                   1
existing_credits          2
job                       1
num_dependents            0
own_telephone             0
foreign_worker            0
label                     0
dtype: int64

In [11]:
data_live.isna().sum(axis='index')

checking_status           1
duration                  0
credit_history            0
purpose                   0
credit_amount             0
savings_status            0
employment                0
installment_commitment    0
personal_status           0
other_parties             0
residence_since           0
property_magnitude        0
age                       0
other_payment_plans       0
housing                   0
existing_credits          2
job                       1
num_dependents            0
own_telephone             0
foreign_worker            0
dtype: int64

# Instructions

- For each data set, split the data into
    - `pandas.DataFrame` of only categorical features
    - `pandas.DataFrame` of only numerical features

- Use `sklearn.impute.SimpleImputer` to handle missing values in each `pandas.DataFrame`
    - Note that these **filling parameters** should be applied to the test data and the live data

- Then, combine the `pandas.DataFrame`s to obtain the cleaned training, test, and live data set
  - Hint: use `.join`
  - Also, verify that the missing values no longer exist

## Handling of missing values for categorical features

In [12]:
data_train_categorical_feature = data_train[categorical_features]
data_train_categorical_feature

checking_status credit_history              purpose    savings_status  \
id                                                                          
495              <0  existing paid  furniture/equipment  no known savings   
296     no checking  existing paid             used car  no known savings   
1          0<=X<200  existing paid             radio/tv              <100   
38            >=200  existing paid   domestic appliance              <100   
568        0<=X<200  existing paid             radio/tv  no known savings   
..              ...            ...                  ...               ...   
836     no checking  existing paid             radio/tv  no known savings   
192        0<=X<200  existing paid             business              <100   
630              <0  existing paid             radio/tv              <100   
560              <0  existing paid             used car  no known savings   
685     no checking  existing paid              new car  no known savings   

    employment     personal_status other_parties property_magnitude  \
id                                                                    
495     1<=X<4        male mar/wid          none                car   
296         <1  female div/dep/mar          none                car   
1       1<=X<4  female div/dep/mar          none        real estate   
38      1<=X<4         male single          none                car   
568     4<=X<7         male single          none                car   
..         ...                 ...           ...                ...   
836     1<=X<4  female div/dep/mar          none                car   
192     1<=X<4         male single          none                car   
630     1<=X<4  female div/dep/mar          none                car   
560        >=7         male single          none  no known property   
685     1<=X<4         male single          none  no known property   

    other_payment_plans   housing      job own_telephone foreign_worker  \
id                                                                        
495                none       own  skilled          none            yes   
296                none      rent  skilled          none            yes   
1                  none       own  skilled          none            yes   
38                 none       own  skilled           yes            yes   
568                none       own  skilled           yes            yes   
..                  ...       ...      ...           ...            ...   
836                none       own  skilled          none            yes   
192                none       own  skilled           yes            yes   
630                none       own  skilled          none            yes   
560                bank  for free  skilled           yes            yes   
685                none  for free  skilled           yes            yes   

     installment_commitment  residence_since  existing_credits  num_dependents  
id                                                                              
495                       2              4.0               1.0               1  
296                       1              4.0               1.0               1  
1                         2              2.0               1.0               1  
38                        2              2.0               1.0               1  
568                       4              1.0               2.0               2  
..                      ...              ...               ...             ...  
836                       4              2.0               1.0               1  
192                       4              2.0               1.0               2  
630                       2              4.0               1.0               1  
560                       4              4.0               1.0               2  
685                       4              4.0               1.0               2  

[748 rows x 17 columns]

In [13]:
data_test_categorical_feature = data_test[categorical_features]
data_test_categorical_feature

checking_status                  credit_history              purpose  \
id                                                                         
271     no checking  critical/other existing credit  furniture/equipment   
726     no checking  critical/other existing credit             radio/tv   
31               <0                   existing paid  furniture/equipment   
343        0<=X<200                   existing paid             business   
605              <0                        all paid  furniture/equipment   
..              ...                             ...                  ...   
75               <0  critical/other existing credit             used car   
454              <0                   existing paid              new car   
468     no checking              delayed previously             business   
386     no checking                   existing paid             radio/tv   
492     no checking  critical/other existing credit             radio/tv   

       savings_status employment     personal_status other_parties  \
id                                                                   
271       500<=X<1000     4<=X<7  female div/dep/mar          none   
726       500<=X<1000     1<=X<4        male mar/wid          none   
31               <100     1<=X<4         male single          none   
343              <100        >=7         male single  co applicant   
605       500<=X<1000     1<=X<4         male single          none   
..                ...        ...                 ...           ...   
75               <100        >=7         male single          none   
454              <100     4<=X<7         male single  co applicant   
468              <100     1<=X<4  female div/dep/mar          none   
386  no known savings         <1  female div/dep/mar          none   
492        100<=X<500     1<=X<4  female div/dep/mar          none   

    property_magnitude other_payment_plans   housing  \
id                                                     
271                car                none      rent   
726     life insurance                none       own   
31                 car              stores       own   
343        real estate                bank       own   
605        real estate              stores       own   
..                 ...                 ...       ...   
75   no known property                none  for free   
454     life insurance                none       own   
468                car                none       own   
386        real estate                none      rent   
492     life insurance                none       own   

                           job own_telephone foreign_worker  \
id                                                            
271                    skilled           yes            yes   
726         unskilled resident          none            yes   
31                     skilled          none            yes   
343  high qualif/self emp/mgmt           yes            yes   
605                    skilled           yes            yes   
..                         ...           ...            ...   
75   high qualif/self emp/mgmt          none            yes   
454                    skilled           yes            yes   
468                    skilled           yes            yes   
386                    skilled           yes            yes   
492                    skilled          none            yes   

     installment_commitment  residence_since  existing_credits  num_dependents  
id                                                                              
271                       3              4.0               1.0               1  
726                       2              2.0               2.0               1  
31                        2              2.0               1.0               1  
343                       1              1.0               1.0               1  
605                       4              4.0               1.0       

In [14]:
data_live_categorical_feature = data_live[categorical_features]
data_live_categorical_feature

checking_status                  credit_history              purpose  \
id                                                                         
271     no checking  critical/other existing credit  furniture/equipment   
726     no checking  critical/other existing credit             radio/tv   
31               <0                   existing paid  furniture/equipment   
343        0<=X<200                   existing paid             business   
605              <0                        all paid  furniture/equipment   
..              ...                             ...                  ...   
725     no checking  critical/other existing credit              new car   
747              <0                   existing paid              new car   
534     no checking                   existing paid             radio/tv   
97         0<=X<200                   existing paid             business   
268              <0                   existing paid              new car   

       savings_status employment     personal_status other_parties  \
id                                                                   
271       500<=X<1000     4<=X<7  female div/dep/mar          none   
726       500<=X<1000     1<=X<4        male mar/wid          none   
31               <100     1<=X<4         male single          none   
343              <100        >=7         male single  co applicant   
605       500<=X<1000     1<=X<4         male single          none   
..                ...        ...                 ...           ...   
725            >=1000     1<=X<4  female div/dep/mar          none   
747              <100         <1  female div/dep/mar          none   
534  no known savings         <1         male single          none   
97         100<=X<500     1<=X<4         male single          none   
268              <100        >=7        male div/sep          none   

    property_magnitude other_payment_plans housing                        job  \
id                                                                              
271                car                none    rent                    skilled   
726     life insurance                none     own         unskilled resident   
31                 car              stores     own                    skilled   
343        real estate                bank     own  high qualif/self emp/mgmt   
605        real estate              stores     own                    skilled   
..                 ...                 ...     ...                        ...   
725        real estate                bank     own         unskilled resident   
747        real estate                none     own         unskilled resident   
534                car                none     own                    skilled   
97                 car                none     own                    skilled   
268     life insurance                none     own  high qualif/self emp/mgmt   

    own_telephone foreign_worker  installment_commitment  residence_since  \
id                                                                          
271           yes            yes                       3              4.0   
726          none            yes                       2              2.0   
31           none            yes                       2              2.0   
343           yes            yes                       1              1.0   
605           yes            yes                       4              4.0   
..            ...            ...                     ...              ...   
725          none            yes                       2              2.0   
747          none            yes                       3              1.0   
534          none            yes                       4              2.0   
97           none            yes                       4              4.0   
268           yes             no                       1              4.0   

     existing_credits  num_dependents  
id      

In [15]:
categorical_imputer = sklearn.impute.SimpleImputer(strategy='most_frequent')
categorical_imputer

SimpleImputer(strategy='most_frequent')

In [16]:
categorical_imputer.fit(data_train_categorical_feature)

SimpleImputer(strategy='most_frequent')

In [17]:
data_train_categorical_feature = pandas.DataFrame(
    categorical_imputer.transform(data_train_categorical_feature),
    columns = categorical_imputer.get_feature_names_out(),
    index = data_train_categorical_feature.index
)
data_train_categorical_feature


checking_status credit_history              purpose    savings_status  \
id                                                                          
495              <0  existing paid  furniture/equipment  no known savings   
296     no checking  existing paid             used car  no known savings   
1          0<=X<200  existing paid             radio/tv              <100   
38            >=200  existing paid   domestic appliance              <100   
568        0<=X<200  existing paid             radio/tv  no known savings   
..              ...            ...                  ...               ...   
836     no checking  existing paid             radio/tv  no known savings   
192        0<=X<200  existing paid             business              <100   
630              <0  existing paid             radio/tv              <100   
560              <0  existing paid             used car  no known savings   
685     no checking  existing paid              new car  no known savings   

    employment     personal_status other_parties property_magnitude  \
id                                                                    
495     1<=X<4        male mar/wid          none                car   
296         <1  female div/dep/mar          none                car   
1       1<=X<4  female div/dep/mar          none        real estate   
38      1<=X<4         male single          none                car   
568     4<=X<7         male single          none                car   
..         ...                 ...           ...                ...   
836     1<=X<4  female div/dep/mar          none                car   
192     1<=X<4         male single          none                car   
630     1<=X<4  female div/dep/mar          none                car   
560        >=7         male single          none  no known property   
685     1<=X<4         male single          none  no known property   

    other_payment_plans   housing      job own_telephone foreign_worker  \
id                                                                        
495                none       own  skilled          none            yes   
296                none      rent  skilled          none            yes   
1                  none       own  skilled          none            yes   
38                 none       own  skilled           yes            yes   
568                none       own  skilled           yes            yes   
..                  ...       ...      ...           ...            ...   
836                none       own  skilled          none            yes   
192                none       own  skilled           yes            yes   
630                none       own  skilled          none            yes   
560                bank  for free  skilled           yes            yes   
685                none  for free  skilled           yes            yes   

    installment_commitment residence_since existing_credits num_dependents  
id                                                                          
495                      2             4.0              1.0              1  
296                      1             4.0              1.0              1  
1                        2             2.0              1.0              1  
38                       2             2.0              1.0              1  
568                      4             1.0              2.0              2  
..                     ...             ...              ...            ...  
836                      4             2.0              1.0              1  
192                      4             2.0              1.0              2  
630                      2             4.0              1.0              1  
560                      4             4.0              1.0              2  
685                      4             4.0              1.0              2  

[748 rows x 17 columns]

In [18]:
data_test_categorical_feature = pandas.DataFrame(
    categorical_imputer.transform(data_test_categorical_feature),
    columns = categorical_imputer.get_feature_names_out(),
    index = data_test_categorical_feature.index
)
data_test_categorical_feature

checking_status                  credit_history              purpose  \
id                                                                         
271     no checking  critical/other existing credit  furniture/equipment   
726     no checking  critical/other existing credit             radio/tv   
31               <0                   existing paid  furniture/equipment   
343        0<=X<200                   existing paid             business   
605              <0                        all paid  furniture/equipment   
..              ...                             ...                  ...   
75               <0  critical/other existing credit             used car   
454              <0                   existing paid              new car   
468     no checking              delayed previously             business   
386     no checking                   existing paid             radio/tv   
492     no checking  critical/other existing credit             radio/tv   

       savings_status employment     personal_status other_parties  \
id                                                                   
271       500<=X<1000     4<=X<7  female div/dep/mar          none   
726       500<=X<1000     1<=X<4        male mar/wid          none   
31               <100     1<=X<4         male single          none   
343              <100        >=7         male single  co applicant   
605       500<=X<1000     1<=X<4         male single          none   
..                ...        ...                 ...           ...   
75               <100        >=7         male single          none   
454              <100     4<=X<7         male single  co applicant   
468              <100     1<=X<4  female div/dep/mar          none   
386  no known savings         <1  female div/dep/mar          none   
492        100<=X<500     1<=X<4  female div/dep/mar          none   

    property_magnitude other_payment_plans   housing  \
id                                                     
271                car                none      rent   
726     life insurance                none       own   
31                 car              stores       own   
343        real estate                bank       own   
605        real estate              stores       own   
..                 ...                 ...       ...   
75   no known property                none  for free   
454     life insurance                none       own   
468                car                none       own   
386        real estate                none      rent   
492     life insurance                none       own   

                           job own_telephone foreign_worker  \
id                                                            
271                    skilled           yes            yes   
726         unskilled resident          none            yes   
31                     skilled          none            yes   
343  high qualif/self emp/mgmt           yes            yes   
605                    skilled           yes            yes   
..                         ...           ...            ...   
75   high qualif/self emp/mgmt          none            yes   
454                    skilled           yes            yes   
468                    skilled           yes            yes   
386                    skilled           yes            yes   
492                    skilled          none            yes   

    installment_commitment residence_since existing_credits num_dependents  
id                                                                          
271                      3             4.0              1.0              1  
726                      2             2.0              2.0              1  
31                       2             2.0              1.0              1  
343                      1             1.0              1.0              1  
605                      4             4.0              1.0              1  
..              

In [19]:
data_live_categorical_feature = pandas.DataFrame(
    categorical_imputer.transform(data_live_categorical_feature),
    columns = categorical_imputer.get_feature_names_out(),
    index = data_live_categorical_feature.index
)
data_live_categorical_feature

checking_status                  credit_history              purpose  \
id                                                                         
271     no checking  critical/other existing credit  furniture/equipment   
726     no checking  critical/other existing credit             radio/tv   
31               <0                   existing paid  furniture/equipment   
343        0<=X<200                   existing paid             business   
605              <0                        all paid  furniture/equipment   
..              ...                             ...                  ...   
725     no checking  critical/other existing credit              new car   
747              <0                   existing paid              new car   
534     no checking                   existing paid             radio/tv   
97         0<=X<200                   existing paid             business   
268              <0                   existing paid              new car   

       savings_status employment     personal_status other_parties  \
id                                                                   
271       500<=X<1000     4<=X<7  female div/dep/mar          none   
726       500<=X<1000     1<=X<4        male mar/wid          none   
31               <100     1<=X<4         male single          none   
343              <100        >=7         male single  co applicant   
605       500<=X<1000     1<=X<4         male single          none   
..                ...        ...                 ...           ...   
725            >=1000     1<=X<4  female div/dep/mar          none   
747              <100         <1  female div/dep/mar          none   
534  no known savings         <1         male single          none   
97         100<=X<500     1<=X<4         male single          none   
268              <100        >=7        male div/sep          none   

    property_magnitude other_payment_plans housing                        job  \
id                                                                              
271                car                none    rent                    skilled   
726     life insurance                none     own         unskilled resident   
31                 car              stores     own                    skilled   
343        real estate                bank     own  high qualif/self emp/mgmt   
605        real estate              stores     own                    skilled   
..                 ...                 ...     ...                        ...   
725        real estate                bank     own         unskilled resident   
747        real estate                none     own         unskilled resident   
534                car                none     own                    skilled   
97                 car                none     own                    skilled   
268     life insurance                none     own  high qualif/self emp/mgmt   

    own_telephone foreign_worker installment_commitment residence_since  \
id                                                                        
271           yes            yes                      3             4.0   
726          none            yes                      2             2.0   
31           none            yes                      2             2.0   
343           yes            yes                      1             1.0   
605           yes            yes                      4             4.0   
..            ...            ...                    ...             ...   
725          none            yes                      2             2.0   
747          none            yes                      3             1.0   
534          none            yes                      4             2.0   
97           none            yes                      4             4.0   
268           yes             no                      1             4.0   

    existing_credits num_dependents  
id                                  

In [20]:
data_train_categorical_feature.isna().sum(axis = 'index')

checking_status           0
credit_history            0
purpose                   0
savings_status            0
employment                0
personal_status           0
other_parties             0
property_magnitude        0
other_payment_plans       0
housing                   0
job                       0
own_telephone             0
foreign_worker            0
installment_commitment    0
residence_since           0
existing_credits          0
num_dependents            0
dtype: int64

In [21]:
data_test_categorical_feature.isna().sum(axis = 'index')

checking_status           0
credit_history            0
purpose                   0
savings_status            0
employment                0
personal_status           0
other_parties             0
property_magnitude        0
other_payment_plans       0
housing                   0
job                       0
own_telephone             0
foreign_worker            0
installment_commitment    0
residence_since           0
existing_credits          0
num_dependents            0
dtype: int64

In [22]:
data_live_categorical_feature.isna().sum(axis = 'index')

checking_status           0
credit_history            0
purpose                   0
savings_status            0
employment                0
personal_status           0
other_parties             0
property_magnitude        0
other_payment_plans       0
housing                   0
job                       0
own_telephone             0
foreign_worker            0
installment_commitment    0
residence_since           0
existing_credits          0
num_dependents            0
dtype: int64

## Handling of missing values for numerical features

In [23]:
numerical_features = [
    'duration',
    'credit_amount',
    'age',
]

In [24]:
data_train_numerical_features = data_train[numerical_features]
data_train_numerical_features

duration  credit_amount  age
id                               
495      24.0         2996.0   20
296      12.0         4675.0   20
1        48.0         5951.0   22
38       10.0         1225.0   37
568      48.0         3979.0   41
..        ...            ...  ...
836      12.0          886.0   21
192      27.0         3915.0   36
630      24.0         3660.0   28
560      24.0         2964.0   49
685      60.0         6527.0   34

[748 rows x 3 columns]

In [25]:
data_test_numerical_features = data_test[numerical_features]
data_test_numerical_features

duration  credit_amount  age
id                               
271      12.0         1402.0   37
726      15.0         1316.0   47
31       24.0         4020.0   27
343      18.0         4439.0   33
605      24.0         2828.0   22
..        ...            ...  ...
75       12.0         1526.0   66
454      24.0         4817.0   31
468      33.0         2764.0   26
386      18.0         1126.0   21
492       6.0         1237.0   27

[250 rows x 3 columns]

In [26]:
data_live_numerical_features = data_live[numerical_features]
data_live_numerical_features

duration  credit_amount  age
id                               
271      12.0         1402.0   37
726      15.0         1316.0   47
31       24.0         4020.0   27
343      18.0         4439.0   33
605      24.0         2828.0   22
..        ...            ...  ...
725       6.0          250.0   41
747      12.0         1274.0   37
534      24.0         3105.0   25
97       18.0         2622.0   34
268      14.0         8978.0   45

[100 rows x 3 columns]

In [27]:
numerical_imputer = sklearn.impute.SimpleImputer(strategy ='mean')
numerical_imputer

SimpleImputer()

In [28]:
numerical_imputer.fit(data_train_numerical_features)

SimpleImputer()

In [29]:
data_train_numerical_features = pandas.DataFrame(
    data = numerical_imputer.transform(data_train_numerical_features),
    columns = numerical_imputer.get_feature_names_out(),
    index = data_train_numerical_features.index
)
data_train_numerical_features

duration  credit_amount   age
id                                
495      24.0         2996.0  20.0
296      12.0         4675.0  20.0
1        48.0         5951.0  22.0
38       10.0         1225.0  37.0
568      48.0         3979.0  41.0
..        ...            ...   ...
836      12.0          886.0  21.0
192      27.0         3915.0  36.0
630      24.0         3660.0  28.0
560      24.0         2964.0  49.0
685      60.0         6527.0  34.0

[748 rows x 3 columns]

In [30]:
data_test_numerical_features = pandas.DataFrame(
    data = numerical_imputer.transform(data_test_numerical_features),
    columns = numerical_imputer.get_feature_names_out(),
    index = data_test_numerical_features.index
)
data_train_numerical_features

duration  credit_amount   age
id                                
495      24.0         2996.0  20.0
296      12.0         4675.0  20.0
1        48.0         5951.0  22.0
38       10.0         1225.0  37.0
568      48.0         3979.0  41.0
..        ...            ...   ...
836      12.0          886.0  21.0
192      27.0         3915.0  36.0
630      24.0         3660.0  28.0
560      24.0         2964.0  49.0
685      60.0         6527.0  34.0

[748 rows x 3 columns]

In [31]:
data_live_numerical_features = pandas.DataFrame(
    data = numerical_imputer.transform(data_live_numerical_features),
    columns = numerical_imputer.get_feature_names_out(),
    index = data_live_numerical_features.index
)
data_live_numerical_features

duration  credit_amount   age
id                                
271      12.0         1402.0  37.0
726      15.0         1316.0  47.0
31       24.0         4020.0  27.0
343      18.0         4439.0  33.0
605      24.0         2828.0  22.0
..        ...            ...   ...
725       6.0          250.0  41.0
747      12.0         1274.0  37.0
534      24.0         3105.0  25.0
97       18.0         2622.0  34.0
268      14.0         8978.0  45.0

[100 rows x 3 columns]

In [32]:
data_train_numerical_features.isna().sum(axis='index')

duration         0
credit_amount    0
age              0
dtype: int64

In [33]:
data_test_numerical_features.isna().sum(axis='index')

duration         0
credit_amount    0
age              0
dtype: int64

In [34]:
data_live_numerical_features.isna().sum(axis='index')

duration         0
credit_amount    0
age              0
dtype: int64

## Reconstruction of data sets
- Combine the 3 `pandas.DataFrame` to obtain the cleaned training, test, and live data set
  - Hint: categoical features, numerical features, and label

In [35]:
data_train =(
    data_train_categorical_feature
    .join(data_train_numerical_features)
    .join(data_train['label'])
)
data_train

checking_status credit_history              purpose    savings_status  \
id                                                                          
495              <0  existing paid  furniture/equipment  no known savings   
296     no checking  existing paid             used car  no known savings   
1          0<=X<200  existing paid             radio/tv              <100   
38            >=200  existing paid   domestic appliance              <100   
568        0<=X<200  existing paid             radio/tv  no known savings   
..              ...            ...                  ...               ...   
836     no checking  existing paid             radio/tv  no known savings   
192        0<=X<200  existing paid             business              <100   
630              <0  existing paid             radio/tv              <100   
560              <0  existing paid             used car  no known savings   
685     no checking  existing paid              new car  no known savings   

    employment     personal_status other_parties property_magnitude  \
id                                                                    
495     1<=X<4        male mar/wid          none                car   
296         <1  female div/dep/mar          none                car   
1       1<=X<4  female div/dep/mar          none        real estate   
38      1<=X<4         male single          none                car   
568     4<=X<7         male single          none                car   
..         ...                 ...           ...                ...   
836     1<=X<4  female div/dep/mar          none                car   
192     1<=X<4         male single          none                car   
630     1<=X<4  female div/dep/mar          none                car   
560        >=7         male single          none  no known property   
685     1<=X<4         male single          none  no known property   

    other_payment_plans   housing  ... own_telephone foreign_worker  \
id                                 ...                                
495                none       own  ...          none            yes   
296                none      rent  ...          none            yes   
1                  none       own  ...          none            yes   
38                 none       own  ...           yes            yes   
568                none       own  ...           yes            yes   
..                  ...       ...  ...           ...            ...   
836                none       own  ...          none            yes   
192                none       own  ...           yes            yes   
630                none       own  ...          none            yes   
560                bank  for free  ...           yes            yes   
685                none  for free  ...           yes            yes   

    installment_commitment residence_since existing_credits num_dependents  \
id                                                                           
495                      2             4.0              1.0              1   
296                      1             4.0              1.0              1   
1                        2             2.0              1.0              1   
38                       2             2.0              1.0              1   
568                      4             1.0              2.0              2   
..                     ...             ...              ...            ...   
836                      4             2.0              1.0              1   
192                      4             2.0              1.0              2   
630                      2             4.0              1.0              1   
560                      4             4.0              1.0              2   
685                      4             4.0              1.0              2   

    duration  credit_amount   age  label  
id                                        
495     24.0         2996.0  20.0    bad  
296     12

In [36]:
data_test =(
    data_test_categorical_feature
    .join(data_test_numerical_features)
    .join(data_test['label'])
)
data_test

checking_status                  credit_history              purpose  \
id                                                                         
271     no checking  critical/other existing credit  furniture/equipment   
726     no checking  critical/other existing credit             radio/tv   
31               <0                   existing paid  furniture/equipment   
343        0<=X<200                   existing paid             business   
605              <0                        all paid  furniture/equipment   
..              ...                             ...                  ...   
75               <0  critical/other existing credit             used car   
454              <0                   existing paid              new car   
468     no checking              delayed previously             business   
386     no checking                   existing paid             radio/tv   
492     no checking  critical/other existing credit             radio/tv   

       savings_status employment     personal_status other_parties  \
id                                                                   
271       500<=X<1000     4<=X<7  female div/dep/mar          none   
726       500<=X<1000     1<=X<4        male mar/wid          none   
31               <100     1<=X<4         male single          none   
343              <100        >=7         male single  co applicant   
605       500<=X<1000     1<=X<4         male single          none   
..                ...        ...                 ...           ...   
75               <100        >=7         male single          none   
454              <100     4<=X<7         male single  co applicant   
468              <100     1<=X<4  female div/dep/mar          none   
386  no known savings         <1  female div/dep/mar          none   
492        100<=X<500     1<=X<4  female div/dep/mar          none   

    property_magnitude other_payment_plans   housing  ... own_telephone  \
id                                                    ...                 
271                car                none      rent  ...           yes   
726     life insurance                none       own  ...          none   
31                 car              stores       own  ...          none   
343        real estate                bank       own  ...           yes   
605        real estate              stores       own  ...           yes   
..                 ...                 ...       ...  ...           ...   
75   no known property                none  for free  ...          none   
454     life insurance                none       own  ...           yes   
468                car                none       own  ...           yes   
386        real estate                none      rent  ...           yes   
492     life insurance                none       own  ...          none   

    foreign_worker installment_commitment residence_since existing_credits  \
id                                                                           
271            yes                      3             4.0              1.0   
726            yes                      2             2.0              2.0   
31             yes                      2             2.0              1.0   
343            yes                      1             1.0              1.0   
605            yes                      4             4.0              1.0   
..             ...                    ...             ...              ...   
75             yes                      4             4.0              2.0   
454            yes                      2             3.0              1.0   
468            yes                      2             2.0              2.0   
386            yes                      4             2.0              1.0   
492            yes                      1             1.0              2.0   

    num_dependents duration  credit_amount   age  label  
id                                                     

In [37]:
data_live =(
    data_live_categorical_feature
    .join(data_live_numerical_features)
)
data_live

checking_status                  credit_history              purpose  \
id                                                                         
271     no checking  critical/other existing credit  furniture/equipment   
726     no checking  critical/other existing credit             radio/tv   
31               <0                   existing paid  furniture/equipment   
343        0<=X<200                   existing paid             business   
605              <0                        all paid  furniture/equipment   
..              ...                             ...                  ...   
725     no checking  critical/other existing credit              new car   
747              <0                   existing paid              new car   
534     no checking                   existing paid             radio/tv   
97         0<=X<200                   existing paid             business   
268              <0                   existing paid              new car   

       savings_status employment     personal_status other_parties  \
id                                                                   
271       500<=X<1000     4<=X<7  female div/dep/mar          none   
726       500<=X<1000     1<=X<4        male mar/wid          none   
31               <100     1<=X<4         male single          none   
343              <100        >=7         male single  co applicant   
605       500<=X<1000     1<=X<4         male single          none   
..                ...        ...                 ...           ...   
725            >=1000     1<=X<4  female div/dep/mar          none   
747              <100         <1  female div/dep/mar          none   
534  no known savings         <1         male single          none   
97         100<=X<500     1<=X<4         male single          none   
268              <100        >=7        male div/sep          none   

    property_magnitude other_payment_plans housing                        job  \
id                                                                              
271                car                none    rent                    skilled   
726     life insurance                none     own         unskilled resident   
31                 car              stores     own                    skilled   
343        real estate                bank     own  high qualif/self emp/mgmt   
605        real estate              stores     own                    skilled   
..                 ...                 ...     ...                        ...   
725        real estate                bank     own         unskilled resident   
747        real estate                none     own         unskilled resident   
534                car                none     own                    skilled   
97                 car                none     own                    skilled   
268     life insurance                none     own  high qualif/self emp/mgmt   

    own_telephone foreign_worker installment_commitment residence_since  \
id                                                                        
271           yes            yes                      3             4.0   
726          none            yes                      2             2.0   
31           none            yes                      2             2.0   
343           yes            yes                      1             1.0   
605           yes            yes                      4             4.0   
..            ...            ...                    ...             ...   
725          none            yes                      2             2.0   
747          none            yes                      3             1.0   
534          none            yes                      4             2.0   
97           none            yes                      4             4.0   
268           yes             no                      1             4.0   

    existing_credits num_dependents  duration  credit_amount   age  
id   

# Instructions
- Create 4 `pandas.DataFrame`s
    - `pandas.DataFrame` of only numerical features. Also, apply `sklearn.preprocessing.StandardScaler`.
    - `pandas.DataFrame` of only ordinal features encoded using `sklearn.preprocessing.OrdinalEncoder`
    - `pandas.DataFrame` of only nominal features encoded using `sklearn.preprocessing.OneHotEncoder`
    - `pandas.DataFrame` of labels encoded using `sklearn.preprocessing.OrdinalEncoder` 
       - Map {`good`, `bad`} to {0, 1} respectively
- Note that these **encoding parameters** should be applied to the test data and the live data as well
- Then, combine the 4 `pandas.DataFrame` to obtain the fully processed training, test, and live data
  - Hint: use `.join`


## For ordinal features
- Define a subset of ordinal features
- Define a list of categorical maps which sort the feature values in appropriate orders
- Use `sklearn.preprocessing.OrdinalEncoder` to create a categorical encoder, then fit using the training data
- Create a new `pandas.DataFrame` of only ordinally encoded features for  training, test, and live data

In [38]:
ordinal_features = [
    'employment', 
    'own_telephone',
    'foreign_worker',
    'installment_commitment',
    'residence_since',
    'existing_credits',
    'num_dependents',
]

In [39]:
ordinal_mapping = [
    ['unemployed', '<1', '1<=X<4', '4<=X<7', '>=7'],
    ['none', 'yes'],
    ['no', 'yes'],
    [1, 2, 3, 4],
    [1.0, 2.0, 3.0, 4.0],
    [1.0, 2.0, 3.0, 4.0],
    [1, 2],
]

In [40]:
ordinal_encoder = sklearn.preprocessing.OrdinalEncoder(categories=ordinal_mapping)
ordinal_encoder

OrdinalEncoder(categories=[['unemployed', '<1', '1<=X<4', '4<=X<7', '>=7'],
                           ['none', 'yes'], ['no', 'yes'], [1, 2, 3, 4],
                           [1.0, 2.0, 3.0, 4.0], [1.0, 2.0, 3.0, 4.0], [1, 2]])

In [41]:
ordinal_encoder.fit(
    X = data_train[ordinal_features]
)

OrdinalEncoder(categories=[['unemployed', '<1', '1<=X<4', '4<=X<7', '>=7'],
                           ['none', 'yes'], ['no', 'yes'], [1, 2, 3, 4],
                           [1.0, 2.0, 3.0, 4.0], [1.0, 2.0, 3.0, 4.0], [1, 2]])

In [42]:
data_train_ordinal_features = pandas.DataFrame(
    data    = ordinal_encoder.transform(data_train[ordinal_features]),
    columns = ordinal_encoder.get_feature_names_out(),
    index   = data_train.index
)
data_train_ordinal_features

employment  own_telephone  foreign_worker  installment_commitment  \
id                                                                       
495         2.0            0.0             1.0                     1.0   
296         1.0            0.0             1.0                     0.0   
1           2.0            0.0             1.0                     1.0   
38          2.0            1.0             1.0                     1.0   
568         3.0            1.0             1.0                     3.0   
..          ...            ...             ...                     ...   
836         2.0            0.0             1.0                     3.0   
192         2.0            1.0             1.0                     3.0   
630         2.0            0.0             1.0                     1.0   
560         4.0            1.0             1.0                     3.0   
685         2.0            1.0             1.0                     3.0   

     residence_since  existing_credits  num_dependents  
id                                                      
495              3.0               0.0             0.0  
296              3.0               0.0             0.0  
1                1.0               0.0             0.0  
38               1.0               0.0             0.0  
568              0.0               1.0             1.0  
..               ...               ...             ...  
836              1.0               0.0             0.0  
192              1.0               0.0             1.0  
630              3.0               0.0             0.0  
560              3.0               0.0             1.0  
685              3.0               0.0             1.0  

[748 rows x 7 columns]

In [43]:
data_test_ordinal_features = pandas.DataFrame(
    data    = ordinal_encoder.transform(data_test[ordinal_features]),
    columns = ordinal_encoder.get_feature_names_out(),
    index   = data_test.index
)
data_test_ordinal_features

employment  own_telephone  foreign_worker  installment_commitment  \
id                                                                       
271         3.0            1.0             1.0                     2.0   
726         2.0            0.0             1.0                     1.0   
31          2.0            0.0             1.0                     1.0   
343         4.0            1.0             1.0                     0.0   
605         2.0            1.0             1.0                     3.0   
..          ...            ...             ...                     ...   
75          4.0            0.0             1.0                     3.0   
454         3.0            1.0             1.0                     1.0   
468         2.0            1.0             1.0                     1.0   
386         1.0            1.0             1.0                     3.0   
492         2.0            0.0             1.0                     0.0   

     residence_since  existing_credits  num_dependents  
id                                                      
271              3.0               0.0             0.0  
726              1.0               1.0             0.0  
31               1.0               0.0             0.0  
343              0.0               0.0             0.0  
605              3.0               0.0             0.0  
..               ...               ...             ...  
75               3.0               1.0             0.0  
454              2.0               0.0             0.0  
468              1.0               1.0             0.0  
386              1.0               0.0             0.0  
492              0.0               1.0             0.0  

[250 rows x 7 columns]

In [44]:
data_live_ordinal_features = pandas.DataFrame(
    data    = ordinal_encoder.transform(data_live[ordinal_features]),
    columns = ordinal_encoder.get_feature_names_out(),
    index   = data_live.index
)
data_live_ordinal_features

employment  own_telephone  foreign_worker  installment_commitment  \
id                                                                       
271         3.0            1.0             1.0                     2.0   
726         2.0            0.0             1.0                     1.0   
31          2.0            0.0             1.0                     1.0   
343         4.0            1.0             1.0                     0.0   
605         2.0            1.0             1.0                     3.0   
..          ...            ...             ...                     ...   
725         2.0            0.0             1.0                     1.0   
747         1.0            0.0             1.0                     2.0   
534         1.0            0.0             1.0                     3.0   
97          2.0            0.0             1.0                     3.0   
268         4.0            1.0             0.0                     0.0   

     residence_since  existing_credits  num_dependents  
id                                                      
271              3.0               0.0             0.0  
726              1.0               1.0             0.0  
31               1.0               0.0             0.0  
343              0.0               0.0             0.0  
605              3.0               0.0             0.0  
..               ...               ...             ...  
725              1.0               1.0             0.0  
747              0.0               0.0             0.0  
534              1.0               1.0             0.0  
97               3.0               0.0             0.0  
268              3.0               0.0             0.0  

[100 rows x 7 columns]

## For nominal features
- Define a subset of nominal features
- Define a list of nominal maps which sort the feature values in appropriate orders
- Use `sklearn.preprocessing.OneHotEncoder` to create an nominal encoder, then fit using the training data
- Create a new `pandas.DataFrame` of only nominally encoded features for training, test, and live data

In [45]:
nominal_features = [
    'job', 
    'checking_status', 
    'credit_history', 
    'purpose', 
    'savings_status',
    'personal_status', 
    'other_parties', 
    'property_magnitude',
    'other_payment_plans', 
    'housing', 
]

In [46]:
nominal_mapping = [
    [ 'unemp/unskilled non res', 'unskilled resident', 'skilled', 'high qualif/self emp/mgmt'],
    ['no checking', '<0', '0<=X<200', '>=200'],
    ['delayed previously', 'all paid', 'critical/other existing credit', 'existing paid', 'no credits/all paid'],
    ['furniture/equipment', 'business', 'new car', 'used car', 'radio/tv', 'domestic appliance', 'repairs', 'education', 'retraining', 'other'],
    ['no known savings', '<100', '100<=X<500', '500<=X<1000', '>=1000'],
    ['male single', 'female div/dep/mar', 'male mar/wid', 'male div/sep'],
    ['none', 'guarantor', 'co applicant'],
    ['no known property', 'life insurance', 'car', 'real estate'],
    ['none', 'stores', 'bank'],
    ['for free', 'rent', 'own']
]

In [47]:
onehot_encoder = sklearn.preprocessing.OneHotEncoder(categories=nominal_mapping, sparse_output=False)
onehot_encoder

OneHotEncoder(categories=[['unemp/unskilled non res', 'unskilled resident',
                           'skilled', 'high qualif/self emp/mgmt'],
                          ['no checking', '<0', '0<=X<200', '>=200'],
                          ['delayed previously', 'all paid',
                           'critical/other existing credit', 'existing paid',
                           'no credits/all paid'],
                          ['furniture/equipment', 'business', 'new car',
                           'used car', 'radio/tv', 'domestic appliance',
                           'repairs', 'education', 'retraining', 'other'],
                          ['no known savings', '<100', '100<=X<500',
                           '500<=X<1000', '>=1000'],
                          ['male single', 'female div/dep/mar', 'male mar/wid',
                           'male div/sep'],
                          ['none', 'guarantor', 'co applicant'],
                          ['no known property', 'life insurance', 'car',
                           'real estate'],
                          ['none', 'stores', 'bank'],
                          ['for free', 'rent', 'own']],
              sparse_output=False)

In [48]:
onehot_encoder.fit(
    X=data_train[nominal_features]
)

OneHotEncoder(categories=[['unemp/unskilled non res', 'unskilled resident',
                           'skilled', 'high qualif/self emp/mgmt'],
                          ['no checking', '<0', '0<=X<200', '>=200'],
                          ['delayed previously', 'all paid',
                           'critical/other existing credit', 'existing paid',
                           'no credits/all paid'],
                          ['furniture/equipment', 'business', 'new car',
                           'used car', 'radio/tv', 'domestic appliance',
                           'repairs', 'education', 'retraining', 'other'],
                          ['no known savings', '<100', '100<=X<500',
                           '500<=X<1000', '>=1000'],
                          ['male single', 'female div/dep/mar', 'male mar/wid',
                           'male div/sep'],
                          ['none', 'guarantor', 'co applicant'],
                          ['no known property', 'life insurance', 'car',
                           'real estate'],
                          ['none', 'stores', 'bank'],
                          ['for free', 'rent', 'own']],
              sparse_output=False)

In [49]:
onehot_encoder.get_feature_names_out()

array(['job_unemp/unskilled non res', 'job_unskilled resident',
       'job_skilled', 'job_high qualif/self emp/mgmt',
       'checking_status_no checking', 'checking_status_<0',
       'checking_status_0<=X<200', 'checking_status_>=200',
       'credit_history_delayed previously', 'credit_history_all paid',
       'credit_history_critical/other existing credit',
       'credit_history_existing paid',
       'credit_history_no credits/all paid',
       'purpose_furniture/equipment', 'purpose_business',
       'purpose_new car', 'purpose_used car', 'purpose_radio/tv',
       'purpose_domestic appliance', 'purpose_repairs',
       'purpose_education', 'purpose_retraining', 'purpose_other',
       'savings_status_no known savings', 'savings_status_<100',
       'savings_status_100<=X<500', 'savings_status_500<=X<1000',
       'savings_status_>=1000', 'personal_status_male single',
       'personal_status_female div/dep/mar',
       'personal_status_male mar/wid', 'personal_status_male div

In [50]:
data_train_nominal_features = pandas.DataFrame(
    data    = onehot_encoder.transform(data_train[nominal_features]),
    columns = onehot_encoder.get_feature_names_out(),
    index   = data_train.index
)
data_train_nominal_features

job_unemp/unskilled non res  job_unskilled resident  job_skilled  \
id                                                                      
495                          0.0                     0.0          1.0   
296                          0.0                     0.0          1.0   
1                            0.0                     0.0          1.0   
38                           0.0                     0.0          1.0   
568                          0.0                     0.0          1.0   
..                           ...                     ...          ...   
836                          0.0                     0.0          1.0   
192                          0.0                     0.0          1.0   
630                          0.0                     0.0          1.0   
560                          0.0                     0.0          1.0   
685                          0.0                     0.0          1.0   

     job_high qualif/self emp/mgmt  checking_status_no checking  \
id                                                                
495                            0.0                          0.0   
296                            0.0                          1.0   
1                              0.0                          0.0   
38                             0.0                          0.0   
568                            0.0                          0.0   
..                             ...                          ...   
836                            0.0                          1.0   
192                            0.0                          0.0   
630                            0.0                          0.0   
560                            0.0                          0.0   
685                            0.0                          1.0   

     checking_status_<0  checking_status_0<=X<200  checking_status_>=200  \
id                                                                         
495                 1.0                       0.0                    0.0   
296                 0.0                       0.0                    0.0   
1                   0.0                       1.0                    0.0   
38                  0.0                       0.0                    1.0   
568                 0.0                       1.0                    0.0   
..                  ...                       ...                    ...   
836                 0.0                       0.0                    0.0   
192                 0.0                       1.0                    0.0   
630                 1.0                       0.0                    0.0   
560                 1.0                       0.0                    0.0   
685                 0.0                       0.0                    0.0   

     credit_history_delayed previously  credit_history_all paid  ...  \
id                                                               ...   
495                                0.0                      0.0  ...   
296                                0.0                      0.0  ...   
1                                  0.0                      0.0  ...   
38                                 0.0                      0.0  ...   
568                                0.0                      0.0  ...   
..                                 ...                      ...  ...   
836                                0.0                      0.0  ...   
192                                0.0                      0.0  ...   
630                                0.0                      0.0  ...   
560                                0.0                      0.0  ...   
685                                0.0                      0.0  ...   

     property_magnitude_no known property  property_magnitude_life insurance  \
id                                                                             
495                                   0.0                                0.0   
296              

In [51]:
data_test_nominal_features = pandas.DataFrame(
    data    = onehot_encoder.transform(data_test[nominal_features]),
    columns = onehot_encoder.get_feature_names_out(),
    index   = data_test.index
)
data_test_nominal_features

job_unemp/unskilled non res  job_unskilled resident  job_skilled  \
id                                                                      
271                          0.0                     0.0          1.0   
726                          0.0                     1.0          0.0   
31                           0.0                     0.0          1.0   
343                          0.0                     0.0          0.0   
605                          0.0                     0.0          1.0   
..                           ...                     ...          ...   
75                           0.0                     0.0          0.0   
454                          0.0                     0.0          1.0   
468                          0.0                     0.0          1.0   
386                          0.0                     0.0          1.0   
492                          0.0                     0.0          1.0   

     job_high qualif/self emp/mgmt  checking_status_no checking  \
id                                                                
271                            0.0                          1.0   
726                            0.0                          1.0   
31                             0.0                          0.0   
343                            1.0                          0.0   
605                            0.0                          0.0   
..                             ...                          ...   
75                             1.0                          0.0   
454                            0.0                          0.0   
468                            0.0                          1.0   
386                            0.0                          1.0   
492                            0.0                          1.0   

     checking_status_<0  checking_status_0<=X<200  checking_status_>=200  \
id                                                                         
271                 0.0                       0.0                    0.0   
726                 0.0                       0.0                    0.0   
31                  1.0                       0.0                    0.0   
343                 0.0                       1.0                    0.0   
605                 1.0                       0.0                    0.0   
..                  ...                       ...                    ...   
75                  1.0                       0.0                    0.0   
454                 1.0                       0.0                    0.0   
468                 0.0                       0.0                    0.0   
386                 0.0                       0.0                    0.0   
492                 0.0                       0.0                    0.0   

     credit_history_delayed previously  credit_history_all paid  ...  \
id                                                               ...   
271                                0.0                      0.0  ...   
726                                0.0                      0.0  ...   
31                                 0.0                      0.0  ...   
343                                0.0                      0.0  ...   
605                                0.0                      1.0  ...   
..                                 ...                      ...  ...   
75                                 0.0                      0.0  ...   
454                                0.0                      0.0  ...   
468                                1.0                      0.0  ...   
386                                0.0                      0.0  ...   
492                                0.0                      0.0  ...   

     property_magnitude_no known property  property_magnitude_life insurance  \
id                                                                             
271                                   0.0                                0.0   
726              

In [52]:
data_live_nominal_features = pandas.DataFrame(
    data    = onehot_encoder.transform(data_live[nominal_features]),
    columns = onehot_encoder.get_feature_names_out(),
    index   = data_live.index
)
data_live_nominal_features

job_unemp/unskilled non res  job_unskilled resident  job_skilled  \
id                                                                      
271                          0.0                     0.0          1.0   
726                          0.0                     1.0          0.0   
31                           0.0                     0.0          1.0   
343                          0.0                     0.0          0.0   
605                          0.0                     0.0          1.0   
..                           ...                     ...          ...   
725                          0.0                     1.0          0.0   
747                          0.0                     1.0          0.0   
534                          0.0                     0.0          1.0   
97                           0.0                     0.0          1.0   
268                          0.0                     0.0          0.0   

     job_high qualif/self emp/mgmt  checking_status_no checking  \
id                                                                
271                            0.0                          1.0   
726                            0.0                          1.0   
31                             0.0                          0.0   
343                            1.0                          0.0   
605                            0.0                          0.0   
..                             ...                          ...   
725                            0.0                          1.0   
747                            0.0                          0.0   
534                            0.0                          1.0   
97                             0.0                          0.0   
268                            1.0                          0.0   

     checking_status_<0  checking_status_0<=X<200  checking_status_>=200  \
id                                                                         
271                 0.0                       0.0                    0.0   
726                 0.0                       0.0                    0.0   
31                  1.0                       0.0                    0.0   
343                 0.0                       1.0                    0.0   
605                 1.0                       0.0                    0.0   
..                  ...                       ...                    ...   
725                 0.0                       0.0                    0.0   
747                 1.0                       0.0                    0.0   
534                 0.0                       0.0                    0.0   
97                  0.0                       1.0                    0.0   
268                 1.0                       0.0                    0.0   

     credit_history_delayed previously  credit_history_all paid  ...  \
id                                                               ...   
271                                0.0                      0.0  ...   
726                                0.0                      0.0  ...   
31                                 0.0                      0.0  ...   
343                                0.0                      0.0  ...   
605                                0.0                      1.0  ...   
..                                 ...                      ...  ...   
725                                0.0                      0.0  ...   
747                                0.0                      0.0  ...   
534                                0.0                      0.0  ...   
97                                 0.0                      0.0  ...   
268                                0.0                      0.0  ...   

     property_magnitude_no known property  property_magnitude_life insurance  \
id                                                                             
271                                   0.0                                0.0   
726              

## For numerical features
- Use a subset of numerical features
- Apply `sklearn.preprocessing.StandardScaler` to create a standard scaler, then fit using the training data
- Create a new `pandas.DataFrame` of only numerically encoded features for both training and test data

In [53]:
numerical_features = [
    'duration',
    'credit_amount',
    'age',
]

In [54]:
data_train_numerical_features = data_train[numerical_features]
data_train_numerical_features

duration  credit_amount   age
id                                
495      24.0         2996.0  20.0
296      12.0         4675.0  20.0
1        48.0         5951.0  22.0
38       10.0         1225.0  37.0
568      48.0         3979.0  41.0
..        ...            ...   ...
836      12.0          886.0  21.0
192      27.0         3915.0  36.0
630      24.0         3660.0  28.0
560      24.0         2964.0  49.0
685      60.0         6527.0  34.0

[748 rows x 3 columns]

In [55]:
data_test_numerical_features = data_test[numerical_features]
data_test_numerical_features

duration  credit_amount   age
id                                
271      12.0         1402.0  37.0
726      15.0         1316.0  47.0
31       24.0         4020.0  27.0
343      18.0         4439.0  33.0
605      24.0         2828.0  22.0
..        ...            ...   ...
75       12.0         1526.0  66.0
454      24.0         4817.0  31.0
468      33.0         2764.0  26.0
386      18.0         1126.0  21.0
492       6.0         1237.0  27.0

[250 rows x 3 columns]

In [56]:
scaler = sklearn.preprocessing.StandardScaler()

In [57]:
scaler.fit(data_train_numerical_features)

StandardScaler()

In [58]:
data_train_std = pandas.DataFrame(
    data = scaler.transform(data_train_numerical_features),
    columns = scaler.get_feature_names_out(),
    index = data_train_numerical_features.index
)
data_train_std

duration  credit_amount       age
id                                    
495  0.273882      -0.080100 -1.359664
296 -0.724523       0.526828 -1.359664
1    2.270692       0.988078 -1.185843
38  -0.890924      -0.720284  0.117817
568  2.270692       0.275237  0.465460
..        ...            ...       ...
836 -0.724523      -0.842826 -1.272753
192  0.523483       0.252102  0.030907
630  0.273882       0.159924 -0.664379
560  0.273882      -0.091667  1.160745
685  3.269098       1.196292 -0.142915

[748 rows x 3 columns]

In [59]:
data_test_std = pandas.DataFrame(
    data = scaler.transform(data_test_numerical_features),
    columns = scaler.get_feature_names_out(),
    index = data_test_numerical_features.index
)
data_test_std

duration  credit_amount       age
id                                    
271 -0.724523      -0.656302  0.117817
726 -0.474922      -0.687389  0.986924
31   0.273882       0.290057 -0.751289
343 -0.225321       0.441518 -0.229825
605  0.273882      -0.140829 -1.185843
..        ...            ...       ...
75  -0.724523      -0.611478  2.638227
454  0.273882       0.578158 -0.403647
468  1.022686      -0.163964 -0.838200
386 -0.225321      -0.756071 -1.272753
492 -1.223726      -0.715946 -0.751289

[250 rows x 3 columns]

In [60]:
data_live_std = pandas.DataFrame(
    data = scaler.transform(data_live_numerical_features),
    columns = scaler.get_feature_names_out(),
    index = data_live_numerical_features.index
)
data_live_std

duration  credit_amount       age
id                                    
271 -0.724523      -0.656302  0.117817
726 -0.474922      -0.687389  0.986924
31   0.273882       0.290057 -0.751289
343 -0.225321       0.441518 -0.229825
605  0.273882      -0.140829 -1.185843
..        ...            ...       ...
725 -1.223726      -1.072729  0.465460
747 -0.724523      -0.702571  0.117817
534  0.273882      -0.040698 -0.925111
97  -0.225321      -0.215294 -0.142915
268 -0.558122       2.082283  0.813103

[100 rows x 3 columns]

In [61]:
data_live_numerical_features = data_live[numerical_features]
data_live_numerical_features

duration  credit_amount   age
id                                
271      12.0         1402.0  37.0
726      15.0         1316.0  47.0
31       24.0         4020.0  27.0
343      18.0         4439.0  33.0
605      24.0         2828.0  22.0
..        ...            ...   ...
725       6.0          250.0  41.0
747      12.0         1274.0  37.0
534      24.0         3105.0  25.0
97       18.0         2622.0  34.0
268      14.0         8978.0  45.0

[100 rows x 3 columns]

## For labels
- Define a subset of ordinal features
- Use `sklearn.preprocessing.OrdinalEncoder` to create an ordinal encoder, then fit using the training data
  - Map {`good`, `bad`} to {0, 1} respectively
- Create a new `pandas.DataFrame` of ordinally encoded labels for both training and test data

In [62]:
label_encoder = sklearn.preprocessing.OrdinalEncoder(categories=[['good','bad']])
label_encoder

OrdinalEncoder(categories=[['good', 'bad']])

In [63]:
label_encoder.fit(data_train[['label']])

OrdinalEncoder(categories=[['good', 'bad']])

In [64]:
data_train_label = pandas.DataFrame(
    data    = label_encoder.transform(data_train[['label']]),
    columns = ['label'],
    index   = data_train.index
)
data_train_label

label
id        
495    1.0
296    0.0
1      1.0
38     0.0
568    0.0
..     ...
836    0.0
192    1.0
630    0.0
560    0.0
685    0.0

[748 rows x 1 columns]

In [65]:
data_test_label = pandas.DataFrame(
    data    = label_encoder.transform(data_test[['label']]),
    columns = ['label'],
    index   = data_test.index
)
data_test_label

label
id        
271    0.0
726    0.0
31     0.0
343    0.0
605    0.0
..     ...
75     0.0
454    1.0
468    0.0
386    0.0
492    0.0

[250 rows x 1 columns]

## Reconstruction of data sets
- Combine the 4 `pandas.DataFrame` to obtain the fully processed training and test data
- Then, combine the 3 `pandas.DataFrame` to obtain the fully processed live data
  - Hint: use `.join`

In [67]:
data_train = (
data_train_ordinal_features
.join(data_train_std)
.join(data_train_nominal_features)
.join(data_train_label)
)

In [69]:
data_test = (
data_test_ordinal_features
.join(data_test_std)
.join(data_test_nominal_features)
.join(data_test_label)
)

In [71]:
data_live = (
data_live_ordinal_features
.join(data_live_std)
.join(data_live_nominal_features)

)

# Instructions
- Write the train data set to `./data/features.train.csv`
- Write the test data set to `./data/features.test.csv`
- Write the live data set to `./data/features.live.csv`

In [72]:
data_train.to_csv('./data/features.train.csv')
data_test.to_csv('./data/features.test.csv')
data_live.to_csv('./data/features.live.csv')